# Set up Azure SQL Database for Infinite Jest

This is an SQL Notebook, which allows you to separate text and code blocks and save code results. Azure Data Studio supports several languages, referred to as kernels, including SQL, PowerShell, Python, and more.

## Connect to the Azure Database

The following is all achieved in the Azure Data Studio app, not in VS Code.

Open the Connections submenu. In the Azure submenu, make sure that your personal account is connected. Then in Servers, click the New Connection button. Click on the Browse tab, then refresh your personal list of databases and servers. Connect to the appropriate SERVER, then after putting in your credentials, select the appropriate database to connect to.

At the top of the window next to "Attach to," select **Select Connection** \> **Change Connection**. Find and select the database you just created a connection to.

## Manually Create Database Schema - This is not the way to go

The first step in configuring the database is to create tables that data will be added to. The following code snippet will create a Users table and a Jokes table, but you don't need to do this... there is a much easier way.

In [1]:
-- Create Users table

/*
CREATE TABLE [dbo].[Users]
(
	[Id] [int] NOT NULL,
	[Name] [varchar](100) NULL,
)
GO
ALTER TABLE [dbo].[Users] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)
GO


-- Create Jokes table, which is linked by a foreign key to the Users table

CREATE TABLE [dbo].[Jokes]
(
	[Id] [int] NOT NULL,
	[UserId] [int] NOT NULL,
	[SetUp] [text] ,
	[Payoff] [text] ,
	[Created] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL,
	[SysEndTime] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL,
	PERIOD FOR SYSTEM_TIME ([Created], [SysEndTime])
)
GO

ALTER TABLE [dbo].[Jokes] ADD PRIMARY KEY CLUSTERED
(
	[Id] ASC
)
GO

ALTER TABLE [dbo].[Jokes]
ADD CONSTRAINT [FK__Jokes__User]
FOREIGN KEY ([UserId]) REFERENCES [dbo].[Users] ([Id])
GO
*/

: Execution failed due to an unexpected error: 
	Cannot connect to the database due to invalid OwnerUri (Parameter 'OwnerUri')

Total execution time: 00:00:00

## Import Starting Data

When creating a database from scratch, use dummy data from a CSV file as a way to automatically create the tables for you. In Azure Data Studio, be sure to download the SQL Server Import extension. Once that's installed, you can go to the Connections menu, then right-click on the server you want to add data to, then click Import wizard.

In this project, use the Users.csv and Jokes.csv files found in the .github/database folder.

In each import, make sure to check that the Id column is the Primary Key for the newly created tables. After they are both done, run the following snippet to make sure that the tables are related by foreign key.

In [4]:
ALTER TABLE [dbo].[Jokes]
ADD CONSTRAINT [FK__Jokes__User]
FOREIGN KEY ([UserId]) REFERENCES [dbo].[Users] ([Id])
GO

Commands completed successfully.

Total execution time: 00:00:00.042

## Creating a DACPAC for GitHub

GitHub will also need a way to pre-load the database every time it starts. Make sure that the SQL Server Dacpac extension is loaded in the Azure Data Studio. Once installed, go to the Connections menu, then right-click on the database you want to take a snapshot of. Right-click and select "Data-tier Application Wizard." Select the second choice, "Extract a data-tier application..." This has already been done in this project and the Dacpac file has been saved in .github/database/dacpac.

## Part 2: Create necessary tables

### Select a route to monitor

Now that you've added the route information, you can select the route to be a "Monitored Route". This will come in handy if you later choose to monitor multiple routes. For now, you will just add the "Education Hill - Crossroads - Eastgate" route.

In [7]:
-- Create MonitoredRoutes table
CREATE TABLE [dbo].[MonitoredRoutes]
(
	[RouteId] [int] NOT NULL
)
GO
ALTER TABLE [dbo].[MonitoredRoutes] ADD PRIMARY KEY CLUSTERED 
(
	[RouteId] ASC
)
GO
ALTER TABLE [dbo].[MonitoredRoutes] 
ADD CONSTRAINT [FK__MonitoredRoutes__Router] 
FOREIGN KEY ([RouteId]) REFERENCES [dbo].[Routes] ([Id])
GO

-- Monitor the "Education Hill - Crossroads - Eastgate" route
INSERT INTO dbo.[MonitoredRoutes] (RouteId) VALUES (100113);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.279

### Select a GeoFence to monitor

In addition to monitoring specific bus routes, you will want to monitor certain GeoFences so you can ultimately get notified when your bus enters or exits where you are (i.e. the GeoFence). For now, you will add a small GeoFence that represents the area near the "Crossroads" bus stop.

In [8]:
-- Create GeoFences table
CREATE SEQUENCE [dbo].[global]
    AS INT
    START WITH 1
    INCREMENT BY 1
GO
CREATE TABLE [dbo].[GeoFences](
	[Id] [int] NOT NULL,
	[Name] [nvarchar](100) NOT NULL,
	[GeoFence] [geography] NOT NULL
) 
GO
ALTER TABLE [dbo].[GeoFences] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)
GO
ALTER TABLE [dbo].[GeoFences] ADD DEFAULT (NEXT VALUE FOR [dbo].[global]) FOR [Id]
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[GeoFences]
(
	[GeoFence]
) USING GEOGRAPHY_AUTO_GRID 
GO

-- Create a GeoFence
INSERT INTO dbo.[GeoFences] 
	([Name], [GeoFence]) 
VALUES
	('Crossroads', 0xE6100000010407000000B4A78EA822CF4740E8D7539530895EC03837D51CEACE4740E80BFBE630895EC0ECD7DF53EACE4740E81B2C50F0885EC020389F0D03CF4740E99BD2A1F0885EC00CB8BEB203CF4740E9DB04FC23895EC068C132B920CF4740E9DB04FC23895EC0B4A78EA822CF4740E8D7539530895EC001000000020000000001000000FFFFFFFF0000000003);
GO


Commands completed successfully.

(1 row affected)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:01.307

(No column name)
1


### Create table to track activity in the GeoFence

Next, create a system-versioned table to keep track of what activity is currently happening within the GeoFence. This means tracking buses entering, exiting, and staying within a given GeoFence. Another table within that table will serve as a histroical log for all activity.

In [9]:
CREATE TABLE [dbo].[GeoFencesActive] 
(
	[Id] [int] IDENTITY(1,1) NOT NULL PRIMARY KEY CLUSTERED,
	[VehicleId] [int] NOT NULL,
	[DirectionId] [int] NOT NULL,
	[GeoFenceId] [int] NOT NULL,
	[SysStartTime] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL,
	[SysEndTime] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL,
	PERIOD FOR SYSTEM_TIME ([SysStartTime], [SysEndTime])
)
WITH ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = [dbo].[GeoFencesActiveHistory] ) )
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.126

### Create a table to store real-time bus data

You'll need one last table to store the real-time bus data as it comes in.

In [10]:
CREATE TABLE [dbo].[BusData](
	[Id] [int] IDENTITY(1,1) NOT NULL,
	[DirectionId] [int] NOT NULL,
	[RouteId] [int] NOT NULL,
	[VehicleId] [int] NOT NULL,
	[Location] [geography] NOT NULL,
	[TimestampUTC] [datetime2](7) NOT NULL,
	[ReceivedAtUTC] [datetime2](7) NOT NULL
)
GO
ALTER TABLE [dbo].[BusData] ADD DEFAULT (SYSUTCDATETIME()) FOR [ReceivedAtUTC]
GO
ALTER TABLE [dbo].[BusData] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
) 
GO
CREATE NONCLUSTERED INDEX [ix1] ON [dbo].[BusData]
(
	[ReceivedAtUTC] DESC
) 
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[BusData]
(
	[Location]
) USING GEOGRAPHY_AUTO_GRID 
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.327

Confirm you've created the tables with the following.

In [17]:
SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_SCHEMA = 'dbo'

(6 rows affected)

Total execution time: 00:00:00.064

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
bus-db,dbo,Routes,BASE TABLE
bus-db,dbo,MonitoredRoutes,BASE TABLE
bus-db,dbo,GeoFences,BASE TABLE
bus-db,dbo,GeoFencesActive,BASE TABLE
bus-db,dbo,GeoFencesActiveHistory,BASE TABLE
bus-db,dbo,BusData,BASE TABLE
